<a href="https://colab.research.google.com/github/tmckim/materials-sp24-colab/blob/main/lec_demos/lec01/lec01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Allen Institute Visual Behavior 2P Dataset - Plotting Neural Data Demo

### Neural data can be complex and can contain many variables within one dataset. How do we make sense of this information in a meaningful way?

Today, we will plot the calcium signal from neurons recorded while mice performed a visual discrimination task.


This notebook will help us investigate data collected from the [Visual Behavior 2P](https://portal.brain-map.org/explore/circuits/visual-behavior-2p) dataset from the Allen Brain Institute.


____
# Learning Objectives

## At the end of this notebook, you'll be able to:
* Plot calcium imaging data based on experimental conditions
* Examine differences in neuronal responses to familiar and novel images
* Understand differences in data format
* Understand how to use common Python packages for data visualization
* Apply best practices for plotting data
______

## Before you start - Save this notebook!

When you open a new Colab notebook from the WebCampus (like you hopefully did for this one), you cannot save changes. So it's  best to store the Colab notebook in your personal drive `"File > Save a copy in drive..."` **before** you do anything else.

The file will open in a new tab in your web browser, and it is automatically named something like: "**lec01.ipynb**". You can rename this to just the title of the assignment "**lec01.ipynb**". Make sure you do keep an informative name (like the name of the assignment) so that you know which files to submit back to WebCampus for grading! More instructions on this are at the end of the notebook.


**Where does the notebook get saved in Google Drive?**

By default, the notebook will be copied to a folder called “Colab Notebooks” at the root (home directory) of your Google Drive. If you use this for other courses or personal code notebooks, I recommend creating a folder for this course and then moving the assignments AFTER you have completed them.<br>

I also recommend you give the folder where you save your notebooks^ a different name than the folder we create below that will store the notebook resources you need each time you work through a course notebook. This includes any data files you will need, links to the images that appear in the notebook, and the files associated with the autograder for answer checking.<br>
You should select a name other than '**NS499-DataSci-course-materials**'. <br>
This folder gets overwritten with each assignment you work on in the course, so you should **NOT** store your notebooks in this folder that we use for course materials! <br><br>For example, you could create a folder called 'NS499-**notebooks**' or something along those lines. <br>

_______

# Allen Institute Visual Behavior 2P dataset overview
### This dataset consists of neural activity measured with 2-photon calcium imaging in the visual cortex of mice performing an image change detection task. In this task, mice learn to report changes in stimulus identity by licking a spout to earn a water reward.


# Dataset Notes

The entire dataset includes neural and behavioral measurements from:

*   107 mice
*   4787 behavior training sessions
*   704 *in vivo* imaging sessions
*   50,482 cortical cells

The data are openly accessible, and include information about all recorded timeseries, behavioral events, and experimental data in a standard data format: [Neurodata Without Borders (NWB)](https://www.nwb.org/nwb-neurophysiology/).




![](task_image.png)

##### In some sessions, the mice perform the task with familiar images they have seen many times during training. In other sessions, mice perform the task with novel images.
##### During 2-photon imaging sessions, 5% of stimulus presentations are randomly omitted, allowing us to examine the effect of unexpected events on neural activity.
##### The same population of cells is imaged over multiple days with varying sensory and behavioral conditions.



![](experiment_overview.png)

##### Multiple cortical areas and depths were measured concurently in each session, at a sample rate of 11Hz.
##### In the full dataset, data was collected from excitatory and inhibitory neural populations.



![](imaging_overview.png)

#### This example will focus on the activity of the two inhibitory types - VIP and SST neurons.

# Setup Steps

## Step 1: Import Data and Files Needed

In [ ]:
# Setup and add files needed to access gdrive
# If you restart colab, start by re-running this cell first!
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


# We make this folder (dir) for all your course materials this semester! You only need to do this in this first notebook.
# It will exist in your gdrive for future notebooks and assignments
!mkdir -p '/content/gdrive/My Drive/NS499-DataSci-course-materials/'  # Don't change this name! This is where everything gets stored in gdrive, consistent for the semester

# These lines clone (copy) all the files you will need from where I store the code+data for the course (github)
# Second part of the code copies the files to this location and folder in your own gdrive
!git clone https://github.com/tmckim/materials-sp24-colab '/content/gdrive/My Drive/NS499-DataSci-course-materials/materials-sp24-colab/'

# Change directory (cd) into the folder where the resources for this lec are stored in gdrive
%cd /content/gdrive/MyDrive/NS499-DataSci-course-materials/materials-sp24-colab/lec_demos/lec01/

## Step 2: Set up coding environment
Each time we start an analysis in Python, we must import the necessary code packages. If you're running this notebook in Colab, the cells below will install packages into your coding environment -- these are *not* installed on your computer.

### Import common packages
Below, we'll `import` a common selection of packages that will help us analyze and plot our data. We'll also configure the plotting in our notebook.

*   This will ensure that our coding environment has [NumPy](https://numpy.org/), [Pandas](https://pandas.pydata.org/), [Matplotlib](https://matplotlib.org/), and [Seaborn](https://seaborn.pydata.org/) installed.


In [ ]:
# If you restart colab, start by re-running this cell first!
# Import our plotting package from matplotlib
import matplotlib.pyplot as plt

# Specify that all plots will happen inline & in high resolution
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Import pandas for working with databases
import pandas as pd

# Import seaborn for plotting adjustments with matplotlib
import seaborn as sns

# Import numpy
import numpy as np

# Print statement to confirm
print('Packages imported!')


# Import the data file into a `pandas` dataframe.

(parquet is just another file format).

In [ ]:
# Import data
filename = "allen_visual_behavior_2p_change_detection_familiar_novel_image_sets.parquet"
data = pd.read_parquet(filename)

## The data is organized as a `pandas` dataframe
#### Each row contains all data for a given cell on a given trial


In [ ]:
# Select a random sample of 5 rows to display
data.sample(5)

In [ ]:
# Let's check out all of our column names
list(data)

# np.sort(list(data)). # if you want them alphabetical instead of the order they appear

In [ ]:
# Get info about our data- rows by col #s
data.shape


#Available data includes:
*   The cell `trace` aligned to stimulus (or omission) onset in a [-1.25, 1.5] second window around onset time
    *   Cell traces are in units of delta F/F, the change in fluorescence relative to baseline
*   The `trace_timestamps` for each trial, aligned to stimulus or omission onset
*   The `mean_response` on a given trial in a 500ms window after stimulus onset
*   The `baseline_response` on a given trial in a 500ms window before stimulus onset
*   The `image_name` for each trial. Trials where the stimulus was omitted have `image_name` = `omitted`
*   The `image_index` indicates which of the 8 eight images for the session was presented. Should also correspond with `image_name`.
*   The `mean_running_speed` in a 500ms window after stimulus onset
*   The `mean_pupil_area` in a 500ms window after stimulus onset
*   The `response_latency` when the mouse licked after stimulus onset
*   Whether or not the trial was `rewarded`
*   Whether or not the trial `is_change`
*   Whether or not the trial was `omitted`
*   Info about `age_in_days` of the animal
*   Biological `sex` of the animal


#### Cell and session level metadata includes:

*   The `stimulus_presentations_id` indicating the trial number within the session
*   The `cell_specimen_id` which is the unique identifier for each cell (note that a cell can be imaged in multiple sessions; if that's the case, the same cell_specimen_id appears in multiple sessions)
*   The `cre_line` indicating the cell type
  *   `Sst-IRES-Cre` labels SST inhibitory cells
  *   `Vip-IRES-Cre` labels VIP inhibitory cells
  *   `Slc17a7-IRES-Cre` labels excitatory cells (these are excluded for now)
*   The `full_genotype` indicates all information about genetic background of the animal
*   The `reporter_line` indicates reporter genetic background of animal
*   The `driver_line` indicates driver genetic background of animal
*   The `indicator` details the calcium flourescence reporter
*   The `imaging_depth` indicating the cortical depth where the cell was located
*   The `targeted_structure` indicating the cortical area the cell was from
*   The `session_type` indicating the session order and image set
*   The `exposure_level` which tells you whether the image set was familiar or novel
*   The `session_number` corresponds to which familiar and novel imaging context was used
*   The `mouse_id` indicating which mouse the cell came from
*   The `ophys_session_id` indicating the recording day for that trial
*   The `ophys_experiment_id` indicating which imaging plane within the session that the cell came from
*   The `ophys_container_id` which links the same imaging plane recorded across multiple sessions. Cells that are imaged across multiple sessions will have the same `cell_specimen_id`
*   The `behavior_session_id` which indicates the behavior session code for each animal




### More info about our data using `unique`

In [ ]:
# Image types
print('exposure_levels:', data.exposure_level.unique())

In [ ]:
# Same or different image
print('stimulus presentations can be changes:', data.is_change.unique())

In [ ]:
# Omission trials
print('stimulus presentations can be omitted:', data.omitted.unique())

In [ ]:
# Cre lines
print('cre lines (cell types) included in this dataset are:', data.cre_line.unique())

In [ ]:
# Mouse count
print('there are', len(data.mouse_id.unique()), 'mice in this dataset')

In [ ]:
# Session count
print('there are', len(data.ophys_session_id.unique()), 'sessions in this dataset')

In [ ]:
# Cell count
print('there are', len(data.cell_specimen_id.unique()), 'cells in this dataset')

In [ ]:
# Female count
females = data[(data.sex == 'F')]
print('there are', len(females.mouse_id.unique()), 'females in this dataset')

In [ ]:
# Male count
males = data[(data.sex == 'F')]
print('there are', len(males.mouse_id.unique()), 'males in this dataset')

In [ ]:
# Brain region(s) info
print('the brain region(s) in this dataset are:', data.targeted_structure.unique())



# How are VIP and SST cells affected by stimulus novelty?

### Plot the population average change response for familiar and novel images for each cre line

#### **Step 1**: Get trials where the image identity changed, for SST and VIP cells

In [ ]:
sst_data = data[(data.cre_line == 'Sst-IRES-Cre')&(data.is_change == True)]
vip_data = data[(data.cre_line == 'Vip-IRES-Cre')&(data.is_change == True)]

In [ ]:
sst_data.shape

In [ ]:
vip_data.shape

#### **Step 2**: Plot the population average change response of SST cells for familiar and novel images

First, let's think about what data we need from the table. In this case, we want to plot the change in calcium flourescence (y-axis) over time (x-axis). <br>
The two columns with this info are: <br>
`trace`<br>
`trace_timestamps`

Let's take a look at this data before we plot.

In [ ]:
# trace data
trace_sst = sst_data.trace
trace_sst

In [ ]:
# timestamps- aligned to stimulus onset or omission
timestamps_sst = sst_data.trace_timestamps
timestamps_sst

Now that we know what these look like, we need to figure out how to incorporate them into a plot. We will first use `matplotlib`, which we imported as `plt`.

In [ ]:
# Notice that all the timestamps are the same- they should be!
# Everything was aligned (in time) for us, so we only need this set of values once for our x-axis
# We reference the column from the dataset that we want (trace_timestamps), and use .values from pandas to access the values in the first one
timestamps = sst_data.trace_timestamps.values[0]
timestamps

In [ ]:
# Now we want to plot the y-values for each image condition- familiar and novel
fam_sst = sst_data[sst_data.exposure_level=='familiar'].trace.values
nov_sst = sst_data[sst_data.exposure_level=='novel'].trace.values

In [ ]:
fam_sst

In [ ]:
# Plotting population average (all cells) SST response for familiar and novel images
plt.plot(timestamps, np.mean(fam_sst), label='familiar')
plt.plot(timestamps, np.mean(nov_sst), label='novel')


# Plot aesthetics
plt.title('SST population average')
plt.xlabel('Time after change (sec)')
plt.ylabel('dF/F')
plt.legend()
plt.show()

In [ ]:
# Plotting code will generate the same plot as above, just more advanced using loop
timestamps = sst_data.trace_timestamps.values[0]

# Here we can loop through the unique levels of exposure (familiar and novel) to plot each
for exposure_level in sst_data.exposure_level.unique():
  traces = sst_data[sst_data.exposure_level==exposure_level].trace.values
  plt.plot(timestamps, np.mean(traces), label=exposure_level)

# Plot aesthetics
plt.title('SST population average')
plt.xlabel('Time after change (sec)')
plt.ylabel('dF/F')
plt.legend()
plt.show()

#### **Step 3**: Plot the population average change response of VIP cells for familiar and novel images

Similar to above, but now with VIP data instead of SST

In [ ]:
# Notice that all the timestamps are the same- they should be!
# Everything was aligned (in time) for us, so we only need this set of values once for our x-axis
# We reference the column from the dataset that we want (trace_timestamps), and use .values from pandas to access the values in the first one
timestamps_vip = vip_data.trace_timestamps.values[0]
timestamps_vip

In [ ]:
# Now we want to plot the y-values for each image condition- familiar and novel
fam_vip = vip_data[vip_data.exposure_level=='familiar'].trace.values
nov_vip = vip_data[vip_data.exposure_level=='novel'].trace.values

In [ ]:
# Plotting population average (all cells) VIP response for familiar and novel images
plt.plot(timestamps_vip, np.mean(fam_vip), label='familiar')
plt.plot(timestamps_vip, np.mean(nov_vip), label='novel')


# Plot aesthetics
plt.title('VIP population average')
plt.xlabel('Time after change (sec)')
plt.ylabel('dF/F')
plt.legend()
plt.show()

In [ ]:
# Advanced with a loop for the conditions (same plot as above though)

timestamps = vip_data.trace_timestamps.values[0]

# Here we can loop through the unique levels of exposure (familiar and novel) to plot each
for exposure_level in vip_data.exposure_level.unique():
  traces = vip_data[vip_data.exposure_level==exposure_level].trace.values
  plt.plot(timestamps, np.mean(traces), label=exposure_level)

# Plot aesthetics
plt.title('VIP population average')
plt.xlabel('time after change (sec)')
plt.ylabel('dF/F')
plt.legend()
plt.show()

### How do we find cells that were imaged across multiple sessions? How do single cells change depending on the image set?

In [ ]:
# which cells are in more than one session? make a list
cells_in_multiple_sessions = []

for cell_specimen_id in vip_data.cell_specimen_id.unique():
  if len(vip_data[vip_data.cell_specimen_id == cell_specimen_id].ophys_session_id.unique()) > 1:
    cells_in_multiple_sessions.append(cell_specimen_id)

print(cells_in_multiple_sessions)

In [ ]:
# Get list of unique
vip_data.cell_specimen_id.unique()

In [ ]:
# how many cells are there?
len(cells_in_multiple_sessions)

In [ ]:
# Test the first cell id from the list (note is not in the multiple session list)
cell_specimen_id = 1086492391

In [ ]:
# See what appears first before evaluating the length and >1
vip_data[vip_data.cell_specimen_id == cell_specimen_id].ophys_session_id.unique()

In [ ]:
# Now put it all together- does the output make sense?
len(vip_data[vip_data.cell_specimen_id == cell_specimen_id].ophys_session_id.unique()) > 1

In [ ]:
# Now pick the first cell id from the multiple session list
cell_specimen_id = 1086495458

In [ ]:
# Find the data for the single cell
# Find the unique entries of ophys_session_id
# If the length of this is great than 1, it was a repeat cell
vip_data[vip_data.cell_specimen_id == cell_specimen_id].ophys_session_id.unique()

In [ ]:
# Now put it all together- does the output make sense?
len(vip_data[vip_data.cell_specimen_id == cell_specimen_id].ophys_session_id.unique()) > 1

#### Plot and review the data from an example cell in the multiple sessions list

In [ ]:
# this one looks like the population average
example_cell_specimen_id = cells_in_multiple_sessions[15]

# Select the data for our cell of interest
cell_data = vip_data[vip_data.cell_specimen_id == example_cell_specimen_id]
timestamps = cell_data.trace_timestamps.values[0]

# Use a loop to plot for both exposure levels (familiar and novel)
for exposure_level in cell_data.exposure_level.unique():
  mean_trace = cell_data[cell_data.exposure_level == exposure_level].trace.mean()
  plt.plot(timestamps, mean_trace, label=exposure_level)

# Plot aesthetics
plt.title(f'cell_specimen_id:{example_cell_specimen_id}') # new - formatted string literal - don't need to know this, it's extra
plt.xlabel('time after change (sec)')
plt.ylabel('dF/F')
plt.legend()
plt.show()

In [ ]:
# this one does *not* look like the population average
example_cell_specimen_id = cells_in_multiple_sessions[0]

# Select the data for our cell of interest
cell_data = vip_data[vip_data.cell_specimen_id==example_cell_specimen_id]
timestamps = cell_data.trace_timestamps.values[0]

# Use a loop to plot for both exposure levels (familiar and novel)
for exposure_level in cell_data.exposure_level.unique():
  mean_trace = cell_data[cell_data.exposure_level == exposure_level].trace.mean()
  plt.plot(timestamps, mean_trace, label=exposure_level)

# Plot aesthetics
plt.title(f'cell_specimen_id:{example_cell_specimen_id}')
plt.xlabel('time after change (sec)')
plt.ylabel('dF/F')
plt.legend()
plt.show()

### What about trial to trial variability? How does the response of a single cell vary across a session?



In [ ]:
# Pre-selected one- feel free to choose another
example_cell_specimen_id = vip_data[vip_data.exposure_level=='novel'].cell_specimen_id.unique()[7]
example_cell_specimen_id

In [ ]:
# Get all the data from the larger table for this cell id we chose above
cell_data = vip_data[vip_data.cell_specimen_id == example_cell_specimen_id]
cell_data

In [ ]:
# What images were shown? Need to know how many in the list to choose one below
cell_data.image_name.unique()

In [ ]:
# pick a cell from a novel image session
example_cell_specimen_id = vip_data[vip_data.exposure_level=='novel'].cell_specimen_id.unique()[7]

# Select the data for our cell of interest
cell_data = vip_data[vip_data.cell_specimen_id == example_cell_specimen_id]
cell_data = cell_data[(cell_data.image_name == cell_data.image_name.unique()[2])] # pick one unique image from the 8- im061

# This controls the color options
offset = 1 / len(cell_data.stimulus_presentations_id.unique())
color = [0, 0, 0]

# Loop for plotting all of the data for this cell
for i, stimulus_presentations_id in enumerate(cell_data.stimulus_presentations_id.unique()):
  trial_data = cell_data[cell_data.stimulus_presentations_id == stimulus_presentations_id]
  timestamps = trial_data.trace_timestamps.values[0]
  trace = trial_data.trace.values[0]
  plt.plot(timestamps, trace, color = color)
  color = [color[0] + offset, color[1] + offset, color[2] + offset]

# Plot aesthetics
plt.title(f'cell_specimen_id:{example_cell_specimen_id}')
plt.xlabel('time after change (sec)')
plt.ylabel('dF/F')
plt.show()

# How do SST and VIP cells respond when stimuli are omitted?

### Plot the population average response to stimulus omission

#### **Step 1**: Get trials where the stimulus was omitted, for SST and VIP cells

In [ ]:
sst_data_omit = data[(data.cre_line == 'Sst-IRES-Cre')&(data.omitted == True)]
vip_data_omit = data[(data.cre_line == 'Vip-IRES-Cre')&(data.omitted == True)]

Plot the population average omission response of SST cells for familiar and novel images

In [ ]:
# Get the x-axis values for the time
timestamps = sst_data_omit.trace_timestamps.values[0] # trace timestamps are relative to stimulus onset

# Get the y-axis values of dF/F to plot for each exposure level (novel and familiar)
for exposure_level in sst_data_omit.exposure_level.unique():
  traces = sst_data_omit[sst_data_omit.exposure_level == exposure_level].trace.values
  plt.plot(timestamps, np.mean(traces), label=exposure_level)

# Plot aesthetics
plt.title('SST population average')
plt.xlabel('time after omission (sec)')
plt.ylabel('dF/F')
plt.legend()
plt.show()

Plot population average omission response of VIP cells for familiar and novel images

In [ ]:
# Get the x-axis values for the time
timestamps = vip_data_omit.trace_timestamps.values[0]

# Get the y-axis values of dF/F to plot for each exposure level (novela and familiar)
for exposure_level in vip_data_omit.exposure_level.unique():
  traces = vip_data_omit[vip_data_omit.exposure_level == exposure_level].trace.values
  plt.plot(timestamps, np.mean(traces), label=exposure_level)

# Plot aesthetics
plt.title('VIP population average')
plt.xlabel('time after omission (sec)')
plt.ylabel('dF/F')
plt.legend()
plt.show()

Additional questions you could ask about and plot from this dataset: <br><br>
1.   Does the omission response correlate with behavior?

2.   How do the dynamics of image and omission evoked activity change over time during the novel image session?

# Comparing data format: plotting with long format data



<u> **Note on data format**</u>: This dataset is in wide format, where each row would represent a different `cell_id `and all data for that cell would be contained in the columns of a single row. See a smaller image of the table we've been working with below, focusing on the 'dff' column (which has multiple values in it) in the example image below for comparison:

![](data_wide.png)

## Tidy Data
However, sometimes datasets can be easier to work with in long-format, or `tidy` data. <br>
To demonstrate this, let's open another file for a **single mouse** that has been reformatted already.

In [ ]:
# Load the data file - SST_data.csv
single_data = pd.read_csv('SST_data.csv')

# Show the data for review
single_data

You can see that in this dataframe, `trial_id` is repeated across multiple rows of the dataset. So, a single `cell_id` is repeated across the rows. This is most similar to the data we've already been working with in this course.

Review variables (columns) in the dataset. They are similar and a smaller subset from the ones above, with some names just slightly adjusted.

`dF_F` calcium imaging signal (baseline corrected, normalized fluorescence) <br>
`time_from_stim` is the timepoint of each row of data, aligned to an image presentation. Onset = time zero, and the times here span a (-1.25, 1.5) sec window  <br>
`cell_id` id number for the cell <br>
`exposure` whether the image for a trial was familiar or novel  <br>
`trial_id` each image presentation is a separate trial <br>
`omitted` whether a trial had an omitted image <br>
`pupil_area` measured 500ms after stimulus presntation <br>
`mean_response` average dF/F over the 500ms following image presentation <br>

Now, let's take a closer look at the data and do some preliminary exploration to understand what we are working with.

Let's see how many cells are in the dataset.

In [ ]:
# Find the unique entries from the cell_id column of the dataset
# Review: index the dataframe using .COLUMN_NAME to refer to the column that you want
len(single_data.cell_id.unique())

In [ ]:
# Find the unique entries of trial IDs
single_data.trial_id.unique()

Let's look at a single `trial_id` and single `cell_id` combination.

In [ ]:
# select which trial id and cell id from the dataframe
singlecell_trial_data = single_data[(single_data.trial_id == 24) & (single_data.cell_id == 1086500633)]

# Display the data
singlecell_trial_data

## Plotting with Seaborn package

### Step 1: Plotting separate conditions with Seaborn
Recall that we imported **seaborn** as `sns`. First, we will use a function called `sns.lineplot`. We will use this to plot our calcium signal based on familiar and novel trials. **Seaborn** allows us to do this by using `hue` in combination with the name of our variable of interest. <br>

Read what this does and refer to the examples in the `sns.lineplot` [documentation](https://seaborn.pydata.org/generated/seaborn.lineplot.html#seaborn.lineplot).


#### 1.1 Lineplot
Let's plot the data like we did above, but remember this is now trials averaged for just a single mouse.


Note: This may take a few seconds, compared to how long it took for the previous plotting code to run above. There are some statistics being computed in the background, so it takes a bit longer (~10 secs) for the plot to be displayed.

In [ ]:
# Lineplot code
sns.lineplot(data = single_data, x = 'time_from_stim', y = 'dF_F', hue = 'exposure');


Let's review the plot above. How does this output compare to what was produced when we plotted with **matplotlib**?
>Which specific aspects of the plot were added automatically?

#### 1.2 Relplot
Now, let's plot again, but using a different function within **seaborn** to compare. <br>
Review the documentation on `relplot` [here](https://seaborn.pydata.org/tutorial/relational.html#visualizing-statistical-relationships). <br>
Read the first section **[Visualizaing statistical relationships](https://seaborn.pydata.org/tutorial/relational.html#visualizing-statistical-relationships)**, and then scroll below (you can skip the sections leading up to the next one ->) and read **[Emphasizing continuity with line plots](https://seaborn.pydata.org/tutorial/relational.html#emphasizing-continuity-with-line-plots)**.

Let's use the code below to plot again.


In [ ]:
# relplot
sns.relplot(data = single_data, x='time_from_stim', y='dF_F', kind = 'line', hue = 'exposure');



Does this plot look similar to the first one? It should be exactly the same (note: the axes might appear visually stretched, but the data hasn't changed!). <br>

This demonstrates that there may be multiple ways to use functions within packages to produce the same plots and data visualizations!

#### 1.3 Changing Labels
We can also change the names of the labels on our plots. There are a few ways to do this, but the easiest is to interface with **matplotlib** like we did previously.


><b>Task:</b> Review the code and add labels by replacing `<insert_text>`.  Feel free to edit other settings and practice options for using `set_style` and `palette`. Run the code below. <br> We are again plotting the calcium signal over (`dF_F`) over time (`time_from_stim`) by familiar/novel (`exposure`) conditions.

In [ ]:
# Plot with adjusted labels
# Another style option
sns.set_style('ticks')

# Plot with palette and adjust labels/title
sns.lineplot(data = single_data, x = 'time_from_stim', y = 'dF_F', hue = 'exposure', palette = 'flare')

plt.xlabel('Time from image presentation')
plt.ylabel('Calcium Activity')
plt.title('Calcium imaging over time')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0);  # you don't need to know this, but it puts the legend outside the box so it doesn't overlap with data

#### 1.4 Adding multiple condition combinations

In [ ]:
# Code to save figures
# Set the size
plt.figure(figsize=(8,4))

# Plot the data
sns.lineplot(data = single_data, x = 'time_from_stim' , y = 'dF_F', hue = 'exposure', style = 'omitted', palette = 'mako')
plt.xlabel('Time from stim')
plt.ylabel('dF/F')
plt.title('All the combinations')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0);  # you don't need to know this, but it puts the legend outside the box so it doesn't overlap with data

## Step 2: Figure aesthetics
With **seaborn**, there are multiple options for controlling [aesthetics](https://seaborn.pydata.org/tutorial/aesthetics.html#). Review the link to see how we will start use `sns.set_style` and `sns.set_context`.  

Let's also look at the data in a different way, using another variable from the dataset: `mean_response`. What does this variable measure, and how does it relate to the previous one we were using, `dF_F`? <br> <br>

We will also introduce another plot type `catplot` to display individual data points. Review the documentation [here](https://seaborn.pydata.org/tutorial/introduction.html#plots-for-categorical-data).

In [ ]:
# Setup and plot data
# Set style and context
sns.set_style("whitegrid")
sns.set_context('talk')

# Subset of the data (faster display, but entire dataset can be used)
data_sample = single_data.sample(1000)

# Categorical plot
sns.catplot(data = data_sample, x = 'exposure', y = 'mean_response', hue = 'exposure', palette = 'mako');

## Extra Practice

1) Can you figure out how to make a [histogram](https://seaborn.pydata.org/tutorial/distributions.html) of pupil area in **Seaborn**? Use the subset of data below. <br>
2) Create a second plot, but split the histogram by `exposure` to plot? <br>

In [ ]:
# Subset of data to use for this exercise
data_sample = single_data.sample(1000)

In [ ]:
# Setup and plot data
# Set style and context
sns.set_style("dark")
sns.set_context('paper')


# Categorical plot
sns.histplot(data = data_sample x = ...);

In [ ]:
# Setup and plot data
# Set style and context
sns.set_style("dark")
sns.set_context('notebook')


# Categorical plot
sns.histplot(data = data_sample, x = ..., hue = ..., palette = 'mako');

_____

## Saving
Remember to save your notebook before closing.
Choose **Save** (and make sure you've already saved a copy in your drive) from the **File** menu.

-----------

# Technical notes & credits


Much more information can be found in the [Allen Brain Institute whitepaper](https://brainmapportal-live-4cc80a57cd6e400d854-f7fdcae.divio-media.net/filer_public/4e/be/4ebe2911-bd38-4230-86c8-01a86cfd758e/visual_behavior_2p_technical_whitepaper.pdf) as well as in their <a href="http://allensdk.readthedocs.io/en/latest/visual_behavior_optical_physiology.html"> documentation</a>.

This file was developed from the [Allen Institute Notebooks](https://allensdk.readthedocs.io/en/latest/visual_behavior_optical_physiology.html), [Neuromatch Academy](https://compneuro.neuromatch.io/projects/neurons/README.html#allen-institute), and the [Columbia-Neuropythonistas repository](https://github.com/Columbia-Neuropythonistas).